In [1]:
import cv2
from PIL import Image
import numpy as np



In [2]:
def load_image(path, gray=False):
    if gray:
        img = cv2.imread(path)
        return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    else:
        return cv2.imread(path)



In [3]:
# 读取两张需要拼接的图像
img1 = load_image('input1.png')
img2 = load_image('input2.png')

# 创建SIFT对象
sift = cv2.SIFT_create()

# 分别对两幅图像提取关键点和特征描述子
kp1, des1 = sift.detectAndCompute(img1, None)
kp2, des2 = sift.detectAndCompute(img2, None)

# 创建用于匹配的BFMatcher对象
bf = cv2.BFMatcher()

# 使用knnMatch函数实现特征点的匹配
matches = bf.knnMatch(des1, des2, k=2)

# 通过筛选匹配点对，计算透视变换矩阵
good = []
for m, n in matches:
    if m.distance < 0.5 * n.distance:
        good.append(m)

if len(good) > 10:
    src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1, 1, 2)
    dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1, 1, 2)

    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

    # 将图片进行拼接
    result = cv2.warpPerspective(img1, M, (img1.shape[1]+img2.shape[1], img1.shape[0]))
    result[0:img2.shape[0], 0:img2.shape[1]] = img2
    cv2.imwrite('result.jpg', result)
else:
    print("匹配点数量过少！")